In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt 
%matplotlib inline

In [54]:
import numpy as np

In [2]:
import os
os.chdir("../")

In [53]:
import copy

In [3]:
words = open('data/names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [4]:
len(words)

32033

In [5]:
## building vocab of chars & mapping to/from integers

chars = sorted(list(set(''.join(words))))
stoi = {s:i + 1 for i,s in enumerate(chars)}
stoi["."] = 0
itos = {i:s for s,i in stoi.items()}

In [6]:
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [7]:
## Building the data set

block_size = 3 ## context length: How many chars to take to predict the next one?

X, Y = [], []

for w in words[0:5]:
    print(w)
    context = [0] * block_size # -> 1st Iter [0, 0, 0]
    
    for ch in w + ".":

        ix = stoi[ch] ## Encoded values of character
        
        X.append(context) ## 1st Iter [0, 0, 0]
        Y.append(ix) ## 1st Iter [itos[ch]]

        print(''.join(itos[i] for i in context), '---->', itos[ix])
        context = context[1:] + [ix] ## Rolling Operation

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ----> e
..e ----> m
.em ----> m
emm ----> a
mma ----> .
olivia
... ----> o
..o ----> l
.ol ----> i
oli ----> v
liv ----> i
ivi ----> a
via ----> .
ava
... ----> a
..a ----> v
.av ----> a
ava ----> .
isabella
... ----> i
..i ----> s
.is ----> a
isa ----> b
sab ----> e
abe ----> l
bel ----> l
ell ----> a
lla ----> .
sophia
... ----> s
..s ----> o
.so ----> p
sop ----> h
oph ----> i
phi ----> a
hia ----> .


#### Understanding Generated X Shape & Y Shape

1. X shape is (N, Context Size). Where N is number of input samples generated while creating X, Y Pairs and Context Size is defined above

2. Y Shape is (N). For each Input Sample, it contains the next character Output

In [10]:
X.shape, X.dtype, 

(torch.Size([32, 3]), torch.int64)

In [9]:
Y.shape, Y.dtype

(torch.Size([32]), torch.int64)

## Projecting characters into 2 Dimensional Space (Embedding Look Up Table)

Number of Chars: 27

Dimensions: 2

In [15]:
C = torch.randn((27, 2))

### Accessing entries of Look-Up Matrix C

In [16]:
## For 5 encoded char "e"
## itos[5]
C[5]

tensor([-0.8031,  1.2199])

In [17]:
## For 0 encoded char "a"
## itos[5]
C[0]

tensor([ 0.4497, -0.8323])

In [18]:
## Is there a better way to index Look Up Matrix?
## Yes One Hot Encoding
F.one_hot(torch.tensor(5), num_classes=27).float() @ C

tensor([-0.8031,  1.2199])

In [ ]:
## First Layer of Neural Language Model is essentially, look up into Embedding matrices for Provide Input

### Getting values for 5th Row & 0th Row simultaneously
C[[5, 0]]

tensor([[-0.8031,  1.2199],
        [ 0.4497, -0.8323]])

In [30]:
### Getting values for 5th Row , 6th Row & 7th Row ,7th Row, 7th Row simultaneously

C[torch.tensor([5, 6, 7, 7, 7])]

tensor([[-0.8031,  1.2199],
        [ 0.1439,  0.5460],
        [-0.9979,  0.2940],
        [-0.9979,  0.2940],
        [-0.9979,  0.2940]])

In [23]:
X.shape, X.dtype, 

(torch.Size([32, 3]), torch.int64)

### How can I get Embedding Look Up entries for X Input `[N, Context Size]`

In [26]:
## Slicing X Tensor for Single word emma (1st name in data)
X_single_name = X[0:4, :]

In [27]:
X_single_name

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        [ 5, 13, 13]])

In [ ]:
## In each row of X
## Number of encoded chars == context sizr
## For each encodeed chars -> (, 2) shape Look-Up Embedding Table output

## End Shape would look like
## (N, Context_Size, Embedding Dim)
C[X_single_name], C[X_single_name].shape

(tensor([[[ 0.4497, -0.8323],
          [ 0.4497, -0.8323],
          [ 0.4497, -0.8323]],
 
         [[ 0.4497, -0.8323],
          [ 0.4497, -0.8323],
          [-0.8031,  1.2199]],
 
         [[ 0.4497, -0.8323],
          [-0.8031,  1.2199],
          [-0.4271, -1.3293]],
 
         [[-0.8031,  1.2199],
          [-0.4271, -1.3293],
          [-0.4271, -1.3293]]]),
 torch.Size([4, 3, 2]))

In [34]:
X[13, 2]

tensor(1)

In [35]:
C[X][13,2]

tensor([-0.3034,  1.1382])

In [36]:
C[1]

tensor([-0.3034,  1.1382])

In [37]:
## We can embedd directly by passing X into C and it should work!

emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [38]:
### Constructing Hidden Layer

W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [ ]:
### Embedding which is input to Hidden Layer has shape of [N, context, embedding dimension] -> (32, 3, 2)
### While Hidden layer neuron shape is (context * embedding dimension, hidden_dim) -> (6, 100)

### Last 2 dimensions of embedding layer needs to combined together so that shape matches for matmul

### One way to solve is to use torch cat function

torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]], dim=1).shape

In [48]:
torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]], dim=1).shape

torch.Size([32, 6])

In [61]:
### More generic way would be use torch unbind and then use torch cat
### Unbinding is done on dimension where context size is defined

torch.cat(torch.unbind(emb, dim=1), dim=1)

tensor([[ 0.4497, -0.8323,  0.4497, -0.8323,  0.4497, -0.8323],
        [ 0.4497, -0.8323,  0.4497, -0.8323, -0.8031,  1.2199],
        [ 0.4497, -0.8323, -0.8031,  1.2199, -0.4271, -1.3293],
        [-0.8031,  1.2199, -0.4271, -1.3293, -0.4271, -1.3293],
        [-0.4271, -1.3293, -0.4271, -1.3293, -0.3034,  1.1382],
        [ 0.4497, -0.8323,  0.4497, -0.8323,  0.4497, -0.8323],
        [ 0.4497, -0.8323,  0.4497, -0.8323, -0.8305, -0.5323],
        [ 0.4497, -0.8323, -0.8305, -0.5323, -0.4855, -0.1252],
        [-0.8305, -0.5323, -0.4855, -0.1252, -1.9454,  1.0713],
        [-0.4855, -0.1252, -1.9454,  1.0713, -0.5646, -0.7614],
        [-1.9454,  1.0713, -0.5646, -0.7614, -1.9454,  1.0713],
        [-0.5646, -0.7614, -1.9454,  1.0713, -0.3034,  1.1382],
        [ 0.4497, -0.8323,  0.4497, -0.8323,  0.4497, -0.8323],
        [ 0.4497, -0.8323,  0.4497, -0.8323, -0.3034,  1.1382],
        [ 0.4497, -0.8323, -0.3034,  1.1382, -0.5646, -0.7614],
        [-0.3034,  1.1382, -0.5646, -0.7

In [57]:
Input_Embedding_Proj = copy.deepcopy(C[X])

In [59]:
Input_Embedding_Proj.shape

torch.Size([32, 3, 2])

In [60]:
Input_Embedding_Proj.reshape(shape=(32, 6))

tensor([[ 0.4497, -0.8323,  0.4497, -0.8323,  0.4497, -0.8323],
        [ 0.4497, -0.8323,  0.4497, -0.8323, -0.8031,  1.2199],
        [ 0.4497, -0.8323, -0.8031,  1.2199, -0.4271, -1.3293],
        [-0.8031,  1.2199, -0.4271, -1.3293, -0.4271, -1.3293],
        [-0.4271, -1.3293, -0.4271, -1.3293, -0.3034,  1.1382],
        [ 0.4497, -0.8323,  0.4497, -0.8323,  0.4497, -0.8323],
        [ 0.4497, -0.8323,  0.4497, -0.8323, -0.8305, -0.5323],
        [ 0.4497, -0.8323, -0.8305, -0.5323, -0.4855, -0.1252],
        [-0.8305, -0.5323, -0.4855, -0.1252, -1.9454,  1.0713],
        [-0.4855, -0.1252, -1.9454,  1.0713, -0.5646, -0.7614],
        [-1.9454,  1.0713, -0.5646, -0.7614, -1.9454,  1.0713],
        [-0.5646, -0.7614, -1.9454,  1.0713, -0.3034,  1.1382],
        [ 0.4497, -0.8323,  0.4497, -0.8323,  0.4497, -0.8323],
        [ 0.4497, -0.8323,  0.4497, -0.8323, -0.3034,  1.1382],
        [ 0.4497, -0.8323, -0.3034,  1.1382, -0.5646, -0.7614],
        [-0.3034,  1.1382, -0.5646, -0.7